In [1]:
import pandas as pd

from bs4 import BeautifulSoup
import requests

### 1. Data set creation and manipulation

In [113]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = pd.DataFrame(data=table[0])
df

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,Rouge
2,M1B,Scarborough,Malvern
3,M1C,Scarborough,Highland Creek
4,M1C,Scarborough,Rouge Hill
...,...,...,...
282,M9V,Etobicoke,Thistletown
283,M9W,Etobicoke,Northwest
284,M9X,Not assigned,Not assigned
285,M9Y,Not assigned,Not assigned


In [124]:
tdf = torontoNB[torontoNB.Borough !='Not assigned']
tdf
#na = torontoNB[ torontoNB['Borough'] == 'Not assigned' ].index
#torontoNB.drop(na, inplace=True)
#torontoNB

,Postcode,Borough,Neighborhood
1,M1B,Scarborough,Rouge
2,M1B,Scarborough,Malvern
3,M1C,Scarborough,Highland Creek
4,M1C,Scarborough,Rouge Hill
5,M1C,Scarborough,Port Union
...,...,...,...
279,M9V,Etobicoke,Mount Olive
280,M9V,Etobicoke,Silverstone
281,M9V,Etobicoke,South Steeles
282,M9V,Etobicoke,Thistletown


In [122]:
dftt = tdf(['']

In [126]:
ttdf = tdf.groupby('Postcode')['Neighborhood'].apply(', '.join).reset_index
ttdf()

,Postcode,Neighborhood
0,M1B,"Rouge , Malvern"
1,M1C,"Highland Creek , Rouge Hill , Port Union"
2,M1E,"Guildwood , Morningside , West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae
...,...,...
98,M9N,Weston
99,M9P,Westmount
100,M9R,"Kingsview Village , Martin Grove Gardens , Ric..."
101,M9V,"Albion Gardens , Beaumond Heights , Humbergate..."


In [133]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = pd.DataFrame(data=table[0])
df

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


Cleaning the original dataframe

In [134]:
# Dropping not assigneed values from Borough
df = df.drop(df[(df.Borough == "Not assigned")].index)
# Replace Not assigned, and NaN values, of Neighborhoods with Borough value 
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)
df.Neighborhood.fillna(df.Borough, inplace=True)
# Drop duplicate rows
df=df.drop_duplicates()


,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


In [135]:
df.update(df.Neighborhood.loc[lambda x: x.str.contains('title')].str.extract('title=\"([^\"]*)',expand=False))
df.update(df.Borough.loc[ lambda x: x.str.contains('title')].str.extract('title=\"([^\"]*)',expand=False))
df.update(df.Neighborhood.loc[lambda x: x.str.contains('Toronto')].str.replace(", Toronto",""))
df.update(df.Neighborhood.loc[lambda x: x.str.contains('Toronto')].str.replace("\(Toronto\)",""))

Creating the final dataframe that includes unique values for the Postcode column

In [136]:
ndf = pd.DataFrame({'Postcode':df.Postcode.unique()})
ndf['Borough'] = pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in ndf.iterrows())
ndf['Neighborhood'] = pd.Series(list(set(df['Neighborhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in ndf.iterrows())
ndf['Neighborhood'] = ndf['Neighborhood'].apply(lambda x: ', '.join(x))
ndf.dtypes

Postcode        object
Borough         object
Neighborhood    object
dtype: object

In [138]:
ndf.shape

(103, 3)

In [140]:
ndf.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


### 2. Latitude and longitude

### 3. Visualization